# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [98]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [99]:
dataset = pd.read_csv("../data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

In [ ]:
dataset.head()

In [ ]:
dataset.TARGET.value_counts()

1) Los datos están estructurados en una serie de variables obtenidas de clientes del departamento de créditos de un banco. Proporcionan datos relevantes sobre el desempeño de 5960 préstamos con garantía hipotecaria, categorizando los mismos según hayan caído en default o no. 

2) La variable objetivo a predecir `TARGET` es binaria y etiqueta aquellos prestamos que han sido pagados y los que no(default). El valor 0 se corresponde con aquellos que han pagado el préstamo, mientras que si toma valor 1 es porque han incumplido.

3) Para cada solicitante de préstamo se registraron 12 variables de entrada o atributos que se mencionan a continuación.

  
 `LOAN`: Monto de la solicitud de préstamo
 
 `MORTDUE`: Monto adeudado de la hipoteca existente
 
 `VALUE`: Valor de la propiedad actual
 
 `YOJ`: Años en el trabajo actual
 
 `DEROG`: Número de informes importantes derogados 
 
 `DELINQ`: Número de líneas de crédito morosas
 
 `CLAGE`: Antigüedad de la línea comercial más antigua en meses
 
 `NINQ`: Número de líneas de crédito recientes
 
 `CLNO`: Número de líneas de crédito
 
`DEBTINC`: Relación deuda-ingresos

4) Los atributos más determinantes para la predicción son: `LOAN`,  `MORTDUE`, `DELINQ` y `DEBTINC`


## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [ ]:
sns.countplot(dataset.TARGET)
plt.title("Variable TARGET")
plt.show()

La variable objetivo esta muy desbalanceada.

#### Ajustamos el modelo

In [ ]:
model = SGDClassifier(random_state=0)
model.fit(X_train, y_train)

#### Predecimos

In [ ]:
y_train_pred= model.predict(X_train)
y_test_pred= model.predict(X_test)

#### Obtención de métricas

1. Accuracy

In [ ]:
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Test accuracy: {test_acc:0.2}')

2. Precisión

In [ ]:
train_precision=precision_score(y_train, y_train_pred) 
test_precision=precision_score(y_test, y_test_pred)
print(f'Train precision: {train_precision:0.2}')
print(f'Test precision: {test_precision:0.2}')

3. Recall

In [ ]:
train_recall=recall_score(y_train, y_train_pred) 
test_recall=recall_score(y_test, y_test_pred)
print(f'Train recall: {train_recall:0.2}')
print(f'Test recall: {test_recall:0.2}')

4. F1 score

In [ ]:
train_f1=f1_score(y_train, y_train_pred)
test_f1=f1_score(y_test, y_test_pred)
print(f'Train f1 score: {train_f1:0.2}')
print(f'Test fi score: {test_f1:0.2}')

In [ ]:
#Sistematizamos todas las métricas en un cuadro
y_train_pred= model.predict(X_train)
y_test_pred= model.predict(X_test)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_pred_train))

print("TESTEO")
print(classification_report(y_test, y_pred_test))

5. Matriz de confusión

In [ ]:
cm=confusion_matrix(y_test,y_pred_test)

sns.heatmap(cm, annot=True)

plt.title("Confusion matrix")       #Esta matriz es rara, indicaría que no hay casos en que el label sea negativo?
plt.show()

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
##Coment FC: no lo quise borrar por las dudas se pierda algo importante, pero me parece que queda más claro como está desarrollado a partir de la celda de abajo
""""
params={'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
        'alpha':[0.1,0.01,0.001,0.0001],
        'learning_rate': ['optimal','constant','invscaling','adaptive'],
        'eta0': [0.01,0.001,0.0001]}

grid=GridSearchCV(classifier,param_grid=params,cv=5,n_jobs=-1,scoring='accuracy',refit=True)

grid.fit(X_train,y_train)

print("El mejor es: %f con %s" % (grid.best_score_, grid.best_params_))
means_acc= grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means_acc, stds, params):
    print("%f (%f) con: %r" % (mean, stdev, param))
""""

#### Definimos una grilla de parámetros

In [ ]:
params={'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
        'alpha':[0.1,0.01,0.001,0.0001],
        'learning_rate': ['optimal','constant','invscaling','adaptive'],
        'eta0': [0.01,0.001,0.0001]}

#### Hacemos la validación cruzada con el modelo de descenso de gradiente y la grilla de hiperparámetros que queremos probar 

In [ ]:
model = SGDClassifier(random_state=0)

cv = GridSearchCV(model, params, scoring='accuracy', cv=5,refit=True)
cv.fit(X, y);

In [ ]:
results = cv.cv_results_

In [ ]:
df = pd.DataFrame(results)
df[['param_loss', 'param_alpha', "param_learning_rate",'mean_test_score', 'std_test_score', 'rank_test_score']]


In [ ]:
cv.best_estimator_

In [ ]:
cv.best_params_

In [ ]:
cv.best_score_

#### Evaluamos, predecimos y obtenemos las métricas del mejor modelo obtenido a partir de los datos de entrenamiento y test

In [ ]:
best_model=SGDClassifier(alpha=0.1, eta0=0.01, learning_rate='constant', loss='squared_hinge',
              random_state=0)
best_model.fit(X_train, y_train)

In [ ]:
y_train_pred_best= best_model.predict(X_train)
y_test_pred_best= best_model.predict(X_test)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred_best))

print("TESTEO")
print(classification_report(y_test, y_test_pred_best))

In [ ]:
cm=confusion_matrix(y_test,y_test_pred_best)

sns.heatmap(cm, annot=True)

plt.title("Confusion matrix")
plt.show()

Se reajustó el modelo para una variedad de combinaciones de valores específicos para cada uno de los hiperparámetros. Se ajustaron 239 modelos diferentes, y se identificó el mejor modelo a aquel que tiene los siguientes parámetros:

- alpha': 0.1,
- 'eta0': 0.01,
- 'learning_rate': 'constant',
- 'loss': 'squared_hinge'

Este modelo, presentó una accuracy promedio de 0.83333.

En comparación con el modelo estimado empleando los parámetros por default, podríamos decir que los resultados que obtenemos son los mismos. 

**FC: Esto tiene sentido? Justo los parámetros del mejor modelo coinciden con los definidos por default o hay algo mal?**


In [ ]:
SGDClassifier??   #para comprobar que los parámetros por default no son los mismos

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


#### Ajustamos el modelo

In [ ]:
model=DecisionTreeClassifier(random_state=0) 
model.fit(X_train, y_train)

#### Predecimos y obtenemos las métricas

In [ ]:
y_train_pred= model.predict(X_train)
y_test_pred=model.predict(X_test)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("TESTEO")
print(classification_report(y_test, y_test_pred))

In [ ]:
cm=confusion_matrix(y_test,y_test_pred)

sns.heatmap(cm, annot=True)

plt.title("Confusion matrix")
plt.show()

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

#### Definimos una grilla de parámetros

In [ ]:
param_grid = {
    "criterion": ["gini", "entropy"],
    'max_depth': [2, 4, 15],
    "min_samples_leaf": [5, 10, 20]
}

#### Hacemos la validación cruzada con el modelo de descenso de árbol de desición y la grilla de hiperparámetros que queremos probar 

In [ ]:
model = DecisionTreeClassifier(random_state=0)

cv = GridSearchCV(model, param_grid, scoring='accuracy', cv=5)
cv.fit(X, y);

In [ ]:
results = cv.cv_results_

In [ ]:
df = pd.DataFrame(results)
df[['param_criterion', 'param_max_depth', "param_min_samples_leaf",'mean_test_score', 'std_test_score', 'rank_test_score']]

In [ ]:
cv.best_estimator_

In [ ]:
cv.best_params_

#### Evaluamos, predecimos y obtenemos las métricas del mejor modelo obtenido a partir de los datos de entrenamiento y test

In [ ]:
best_model=DecisionTreeClassifier(criterion="gini", max_depth=4, min_samples_leaf=10, random_state=0)
best_model.fit(X_train, y_train)

In [ ]:
y_train_pred_best= best_model.predict(X_train)
y_test_pred_best= best_model.predict(X_test)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred_best))

print("TESTEO")
print(classification_report(y_test, y_test_pred_best))

In [ ]:
cm=confusion_matrix(y_test,y_test_pred_best)

sns.heatmap(cm, annot=True)

plt.title("Confusion matrix")
plt.show()

In [ ]:
plot_tree(best_model);

MODIFICAR
Se reajustó el modelo para una variedad de combinaciones de valores específicos para cada uno de los hiperparámetros. Se ajustaron 17 modelos diferentes, y se identificó el mejor modelo a aquel que tiene los siguientes parámetros:


- 'criterion': 'gini', 
- 'max_depth': 4, 
- 'min_samples_leaf': 10

Este modelo, presentó una accuracy promedio de 0.89.

En comparación con el modelo estimado empleando los parámetros por default, podríamos decir que la accuracy promedio del conjunto de test mejora solo levemente. 
En cuanto a las otras métricas, el mejor modelo encontrado presenta ... 
**(FC: tengo dudas sobre la comparación entre las métricas, cuál debería mirar?)**